In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2
%matplotlib inline

<IPython.core.display.Javascript object>

In [2]:
import os
from pathlib import Path
import pickle

home = str(Path.home())
data_path = os.path.join(home, "GA_generation", "Data", "distributions_along_lines.pkl")
perp_path = os.path.join(home, "GA_generation", "Data", "Normals_sampling.parquet")

import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp
from scipy.spatial.distance import cdist

from traffic.core import Traffic, Flight
from traffic.data import airports
from traffic.drawing import countries
import traffic.core.geodesy as geo

X_along_line = pd.read_pickle(data_path)
perpendiculars = pd.read_parquet(perp_path)
perpendiculars["angle"] = np.arctan(perpendiculars.m)

<ipython-input-2-5bd5767fab0a>:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


<IPython.core.display.Javascript object>

In [3]:
# Signed curvature
def curvature(x, y):
    x_t = np.gradient(x)
    y_t = np.gradient(y)
    xx_t = np.gradient(x_t)
    yy_t = np.gradient(y_t)

    return (xx_t * y_t - x_t * yy_t) / (x_t * x_t + y_t * y_t) ** (3 / 2)


from scipy.spatial import distance


def energy_distance(x, y):
    n1 = x.shape[0]
    n2 = y.shape[0]
    a = cdist(x, y, "euclidean").mean()
    b = cdist(x, x, "euclidean").mean()
    c = cdist(y, y, "euclidean").mean()
    e = (n1 * n2 / (n1 + n2)) * (2 * a - b - c)
    return e

<IPython.core.display.Javascript object>

In [22]:
import openturns as ot
from sklearn import preprocessing
import pyvinecopulib as pv
from scipy.interpolate import CubicSpline

# marginals
sample = ot.Sample(np.array(X_along_line))
dimension = sample.getDimension()
marginalFactories = []
for factory in ot.DistributionFactory.GetContinuousUniVariateFactories():
    if str(factory).startswith("Histogram"):
        # ~ non-parametric
        continue
    marginalFactories.append(factory)
estimated_marginals = [
    ot.FittingTest.BestModelBIC(sample.getMarginal(i), marginalFactories)[0]
    for i in range(dimension)
]
estimated_marginals

copula_sample = ot.Sample(sample.getSize(), sample.getDimension())
copula_sample.setDescription(sample.getDescription())
for index in range(sample.getDimension()):
    copula_sample[:, index] = estimated_marginals[index].computeCDF(sample[:, index])

blocs = []
for i in range(copula_sample.getDimension() - 1):
    blocs.append([i, i + 1])
blocs

copulaFactories = []
for factory in ot.DistributionFactory.GetContinuousMultiVariateFactories():
    if not factory.build().isCopula():
        continue
    copulaFactories.append(factory)

estimated_copulas = [
    ot.FittingTest.BestModelBIC(copula_sample.getMarginal(bloc), copulaFactories)[0]
    for bloc in blocs
]
estimated_copula = ot.ComposedCopula(estimated_copulas)
estimated_distribution = ot.ComposedDistribution(estimated_marginals, estimated_copula)

TypeError: InvalidArgumentException : The distribution collection must have a size equal to the copula dimension

<IPython.core.display.Javascript object>

In [25]:
estimated_copula.getDimension()

58

<IPython.core.display.Javascript object>